In [12]:
import cv2
import numpy as np

imgs = []
src = cv2.imread('img/polygon.bmp')
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

_, obj_bin = cv2.threshold(src_gray, 128, 255, cv2.THRESH_BINARY)
obj_contours, _ = cv2.findContours(obj_bin, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

for c in range(len(obj_contours)):
    area = cv2.contourArea(obj_contours[c])
    if 500 < area < 50000:
        x,y,w,h = cv2.boundingRect(obj_contours[c])
        roi = src[y:y+h, x:x+w]
        #dst = roi.copy()
        dst = np.full((200,200,3), 255, np.uint8)
        dst[20:20+h, 20:20+w] = roi
        imgs.append(dst)
        
feature = cv2.KAZE_create() #특징점 추출 객체 생성   
kp1, desc1 = feature.detectAndCompute(src_gray, None)#특징점 추출 및 디스크립트 계산
dst1 = cv2.drawKeypoints(src, kp1, None,
                         flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow('dst111', dst1)

matcher = cv2.BFMatcher_create()

for i in range(len(imgs)):
    img_gray = cv2.cvtColor(imgs[i], cv2.COLOR_BGR2GRAY)
    img_gray[img_gray[:,:]>150]=255
    
    kp2, desc2 = feature.detectAndCompute(img_gray, None)
    dst2 = cv2.drawKeypoints(img_gray, kp2, None,
                         flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    matches = matcher.match(desc1, desc2)
    matches = sorted(matches, key=lambda x: x.distance)
    good_matches = matches[:10]
    
    dst0 = cv2.drawMatches(src, kp1, imgs[i], kp2, good_matches, None)
    cv2.imshow('dst'+str(i), dst0)
    
cv2.waitKey()
cv2.destroyAllWindows()

In [13]:
import cv2
import numpy as np

imgs = []   #도형한개씩 추출한 이미지들과 그 좌표[[좌표정보, 이미지], [좌표정보, 이미지], [좌표정보, 이미지]]
masks = []  #도형 한개마다 마스크 이미지

def make_pic(pic, num):
    if num<0 or num>=len(imgs):
        print('잘못된 번호')
        return
    imm = imgs[num][1]#배경
    h, w = imm.shape[:2]
    pic = cv2.resize(pic, (w, h))#추가영상
    mask = masks[num]
    mask_inv = cv2.bitwise_not(mask)
    
    fg = cv2.bitwise_and(pic, pic, mask=mask_inv)
    bg = cv2.bitwise_and(imm, imm, mask=mask) 
    res = cv2.add(bg, fg)
    
    return res

src = cv2.imread('img/polygon.bmp')#도형들
lenna = cv2.imread('img/lenna.bmp')#사람사진
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

_, obj_bin = cv2.threshold(src_gray, 128, 255, cv2.THRESH_BINARY)
obj_contours, _ = cv2.findContours(obj_bin, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

for c in range(len(obj_contours)):
    area = cv2.contourArea(obj_contours[c])
    if 500 < area < 50000:
        x,y,w,h = cv2.boundingRect(obj_contours[c])
        roi = src[y:y+h, x:x+w]
        dst = roi.copy()
        p = (x,y,w,h)
        imgs.append([p, dst])
        img_gray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
        img_gray[img_gray[:,:]>150]=255  #배경 회색을 흰색으로 변경
        _, thresh = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        masks.append(thresh)  

for i in range(len(imgs)):
    res = make_pic(lenna, i)
    x,y,w,h = imgs[i][0]
    src_copy = src.copy()
    roi = src_copy[y:y+h, x:x+w]
    roi[:, :, :] = res
    cv2.imshow('dst', src_copy)
    cv2.waitKey(1000)
    
cv2.waitKey()
cv2.destroyAllWindows()

In [18]:
import cv2
import numpy as np
import sys
import math

cam = cv2.VideoCapture(0)

w=700
h=600

ret, fr = cam.read()

if not ret:
    sys.exit()
    
fr = cv2.flip(fr, 1)#좌우 반전
fr = cv2.resize(fr, (w, h))
gray_bg = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)

while True:
    ret, fr = cam.read()
    if ret:
        fr = cv2.flip(fr, 1)#좌우 반전
        fr = cv2.resize(fr, (w, h))
        gray_fg = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(gray_bg, gray_fg, None, 0.5, 3, 15, 3, 5, 1.1, 0)
        vx, vy = flow[..., 0], flow[..., 1]
        mag, ang = cv2.cartToPolar(vx, vy)
        
        hand = np.zeros((h, w), dtype=np.uint8)#프레임과 동일한 크기의 검정 판 이미지 생성
        hand[mag > 2.0]=255  #검정 판에서 mag값이 2.0보다 큰 영역만 흰색으로 수정
        
        mx = cv2.mean(vx, mask=hand)[0]
        my = cv2.mean(vy, mask=hand)[0]
        
        m_mag = math.sqrt(mx*mx+my*my)#피타고라스의 길이 계산
        
        if m_mag > 4.0:
            m_ang = math.atan2(my, mx)*180/math.pi  #각도 계산
            m_ang += 180   #flip()으로 좌우 반전. 180을 더해줌
            
            if m_ang >=45 and m_ang <135:
                print('위쪽')
            elif m_ang >=135 and m_ang <225:
                print('오른쪽')
            elif m_ang >=225 and m_ang <315:
                print('아래쪽')
            else:
                print('왼쪽')
            
        cv2.imshow('img', fr)
        if cv2.waitKey(1)==27:
            break
cam.release()
cv2.destroyAllWindows()

왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
왼쪽
